In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import numpy as np
import pandas as pd

In [1]:
!pip install keras

In [5]:
df = pd.read_csv("Price_Interval.csv")
# Extract 'Date' and 'Interval Range' columns
df_extracted = df[['Date', 'Interval_Range']]
df_extracted.head()

,Date,Interval_Range
0,2013-07-01,14.329286-14.723929
1,2013-07-02,14.623929-15.058214
2,2013-07-03,14.908929-15.106429
3,2013-07-05,14.833929-15.1175
4,2013-07-08,14.666071-15.035714


In [6]:
df_extracted[['Low', 'High']] = df_extracted['Interval_Range'].str.split('-', expand=True)

# Convert 'Low' and 'High' to numeric, and calculate the 'Mid' from 'Low' and 'High'.
df_extracted['Low'] = pd.to_numeric(df_extracted['Low'])
df_extracted['High'] = pd.to_numeric(df_extracted['High'])
df_extracted['Mid'] = (df_extracted['Low'] + df_extracted['High']) / 2

df_extracted = df_extracted.drop(columns=['Interval_Range'])
df_extracted.head()

C:\Users\eq220\AppData\Local\Temp\ipykernel_18548\2003912284.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_extracted[['Low', 'High']] = df_extracted['Interval_Range'].str.split('-', expand=True)
C:\Users\eq220\AppData\Local\Temp\ipykernel_18548\2003912284.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_extracted[['Low', 'High']] = df_extracted['Interval_Range'].str.split('-', expand=True)
C:\Users\eq220\AppData\Local\Temp\ipykernel_18548\2003912284.py:4: SettingWithCopyWarning: 
A value is t

,Date,Low,High,Mid
0,2013-07-01,14.329286,14.723929,14.526608
1,2013-07-02,14.623929,15.058214,14.841071
2,2013-07-03,14.908929,15.106429,15.007679
3,2013-07-05,14.833929,15.117500,14.975714
4,2013-07-08,14.666071,15.035714,14.850893


In [7]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
data = df_extracted

scaled_Low=scaler.fit_transform(np.array(data['Low']).reshape(-1,1))

In [8]:
##splitting dataset into train and test split
def train_test_split_portion(data, train_portion):
    training_size =int(len(data) * train_portion)
    test_size =len(data)-training_size
    train_data,test_data = data[0:training_size,:], data[training_size:len(data),:1]
    return train_data, test_data

In [9]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [10]:
from matplotlib import pyplot as plt


### Plotting
def Ploting_Stacked_LSTMprediction (original_data, scaled_data, train_predict, test_predict, look_back):
    # shift train predictions for plotting
    trainPredictPlot = np.empty_like(scaled_data)
    trainPredictPlot[:, :] = np.nan
    trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict

    # shift test predictions for plotting
    testPredictPlot = np.empty_like(scaled_data)
    testPredictPlot[:, :] = np.nan
    testPredictPlot[len(train_predict)+(look_back*2)+1:len(scaled_data)-1, :] = test_predict

    trainPredictPlot = pd.DataFrame(trainPredictPlot, index = original_data.index)
    testPredictPlot = pd.DataFrame(testPredictPlot, index = original_data.index)
    real = pd.DataFrame(scaler.inverse_transform(scaled_data), index = original_data.index)
    # plot baseline and predictions

    plt.figure(figsize=(18,10))
    plt.plot(real)
    plt.plot(trainPredictPlot, label="Train Predict")
    plt.plot(testPredictPlot, label="Test Predict")
    plt.legend(loc = 'best')
    plt.show()

In [11]:
def plot_Stacked_LSTM_prediction_30days (original_data, scaled_data, test_data, look_back):
    x_input=test_data[len(test_data)-(look_back+30):len(test_data)-30].reshape(1,-1)  # leave 130 day left
    temp_input=list(x_input)
    temp_input=temp_input[0].tolist()

    lst_output=[]
    n_steps=look_back
    i=0
    while(i<30):

        if(len(temp_input)>look_back):
            x_input=np.array(temp_input[1:])
            #print("{} day input {}".format(i,x_input))
            x_input=x_input.reshape(1,-1)
            x_input = x_input.reshape((1, n_steps, 1))

            yhat = model.predict(x_input, verbose=0)
            #print("{} day output {}".format(i,yhat))
            temp_input.extend(yhat[0].tolist())
            temp_input=temp_input[1:]

            lst_output.extend(yhat.tolist())
            i=i+1
        else:
            x_input = x_input.reshape((1, n_steps,1))
            yhat = model.predict(x_input, verbose=0)
            #print(yhat[0])
            temp_input.extend(yhat[0].tolist())
            #print(len(temp_input))
            lst_output.extend(yhat.tolist())
            i=i+1

    df1 = pd.DataFrame(scaler.inverse_transform(scaled_data), index = original_data.index)
    df2 = df1[len(df1)-30:]
    df3 = pd.DataFrame(scaler.inverse_transform(lst_output), index = original_data[len(original_data)-30:].index)
    plt.figure(figsize=(20,10))
    plt.plot(df1[:len(df1)-30])
    plt.plot(df2)
    plt.plot(df3, label = 'Prediction')
    plt.legend(loc = 'best')
    #testscore = math.sqrt(mean_squared_error(df2,df3))

    #return testscore